In [1]:
import datetime
import seaborn as sns
import pandas as pd
from etsy_interface import Etsy
import pickle
import time

e = Etsy('zdipkqu8fxsomaoriwj1x98a', 'rxv4sdcs5g') # gotten from signing up at etsy.com/developers

In [8]:
total_listing_data = []
for i in range(0, 500):
    index = 100*i
    result = e.show_listings('100', str(index))
    total_listing_data.append(result)
    if i in range(9,500,10):
        time.sleep(1)



with open("total_listing_data.txt", "wb") as fp:
    pickle.dump(total_listing_data, fp)

/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=0
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=100
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=200
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=300
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=400
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=500
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=600
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=700
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsom

/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=4600
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=4700
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=4800
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=4900
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=5000
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=5100
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=5200
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=5300
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdi

/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=9200
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=9300
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=9400
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=9500
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=9600
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=9700
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=9800
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=9900
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdi

/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=13800
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=13900
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=14000
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=14100
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=14200
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=14300
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=14400
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=14500
/listings/active
https://openapi.etsy.com/v2/listings/active?api

/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=18400
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=18500
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=18600
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=18700
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=18800
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=18900
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=19000
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=19100
/listings/active
https://openapi.etsy.com/v2/listings/active?api

...

/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=27600
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=27700
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=27800
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=27900
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=28000
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=28100
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=28200
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=28300
/listings/active
https://openapi.etsy.com/v2/listings/active?api

/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=32200
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=32300
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=32400
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=32500
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=32600
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=32700
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=32800
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=32900
/listings/active
https://openapi.etsy.com/v2/listings/active?api

/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=36800
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=36900
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=37000
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=37100
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=37200
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=37300
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=37400
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=37500
/listings/active
https://openapi.etsy.com/v2/listings/active?api

/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=41400
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=41500
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=41600
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=41700
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=41800
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=41900
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=42000
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=42100
/listings/active
https://openapi.etsy.com/v2/listings/active?api

/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=46000
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=46100
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=46200
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=46300
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=46400
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=46500
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=46600
/listings/active
https://openapi.etsy.com/v2/listings/active?api_key=zdipkqu8fxsomaoriwj1x98a&limit=100&offset=46700
/listings/active
https://openapi.etsy.com/v2/listings/active?api

In [2]:
with open("total_listing_data.txt", "rb") as fp2:
    total_listing_data_loaded = pickle.load(fp2)


{'listing_id': 587121441, 'state': 'edit', 'sku': []}


In [3]:
user_id_dict = {}
for i in range(0, len(total_listing_data_loaded)):
    for j in range(0, len(total_listing_data_loaded[i]['results'])):
        if 'user_id' not in total_listing_data_loaded[i]['results'][j].keys():
            continue
        key = str(total_listing_data_loaded[i]['results'][j]['user_id'])
        if key not in user_id_dict.keys():
            user_id_dict[key] = 1
len(user_id_dict.keys())


13798

In [8]:
e2 = Etsy('dnbju23vywp4369ccw4ugcut', '5ac0qkonp8')
with open('user_info_temp.txt', "rb") as fp4:
    user_info=pickle.load(fp4)
print(len(user_info.keys()))

13798


In [7]:
counter = 0
for single_key in user_id_dict.keys():
    try:
        if single_key not in user_info.keys():
            key_int=int(single_key)
            element=e2.get_user_info(key_int)
            user_info[single_key] = element
            counter += 1
            if counter % 10 == 0:
                time.sleep(1)
            if counter % 100 == 0:
                filename = 'user_info_' + str(counter) + '.txt'
                with open(filename, "wb") as fp3:
                    pickle.dump(user_info, fp3)
    except:
        print('something wrong')
with open('user_info_complete.txt', "wb") as fp3:
            pickle.dump(user_info, fp3)

https://openapi.etsy.com/v2/users/110312923?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/100252069?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/21446003?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/80378910?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/52031860?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/57579117?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/28086453?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/39919680?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/11806016?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/55567506?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/10520024?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/72471780?api_key=dnbju23vywp4369ccw4ugcut
https://openapi.etsy.com/v2/users/135478507?api_key=dnbju23vywp4369ccw4ugcut
https://o

In [51]:
print(len(user_info.keys()))

9348
